# Topos Assignment
https://docs.google.com/document/d/1AEywYFTjh4RJla8hnBF16G5y-7vDMbF9xHVsyGG8vwo/edit

This Assigment is my attemt to scrape various kinds of data from wikipedia alone for the [cities in united states](https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population) ranked by population.<br>

I added six more coulmns in addition to the existing columns in the table:

- *States Abreviation*
- *City webpage url*
- *% of White Peaople*
- *% of Black Peaople*
- *Latitude*
- *Longitude*

#### I will divide the process into there parts
1. Part I
 
 1. Getting the intial Table
 2. Web scraping city wikilinks
 3. Adding states abbreviations


2. Part II
 
 1. Web scraping and parsing Demographics
 2. Adding location data
 3.


3. Part III
 
 1. Vizualization
 2. Final Thoughts

# Part I

In [1]:
# import all the required libraries
import pandas as pd
import bs4 as bs
import pickle
import requests
import webbrowser
import numpy as np
import re
from urllib.request import urlopen, Request

In [2]:
#using pandas to import all the tables from the wiki page
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')

In [3]:
# function to extract the required data
def find_table(column1,column2,dataset):
    for df in dataset:
        if column1 in list(df.columns) or column2 in list(df.columns):
            return df 

In [4]:
# function to parse required text
def parse_txt(txt, types):
    mats = [] 
    for p in types: 
        pattern = re.compile(p)
        matches = pattern.finditer(txt)
        for match in matches:
            mats.append(match.group(0))
    return mats

In [5]:
data = find_table('City','Location', dfs)

In [6]:
#data = data.set_index('2018rank')

In [7]:
# altering the data
clean = lambda x: x[:x.find('[')] if '[' in x else x
add_ = lambda x: x.replace(' ', '_')
data['City'] = data['City'].apply(clean)
data['State[c]'] = data['State[c]'].apply(add_)
data.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location
0,1,New York City,New_York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W


In [8]:
print('The size of the data is '+str(data.shape))

The size of the data is (314, 11)


<br>

**Leveraging the power of beautiful soup to extract wikipedia links for the cities**


In [9]:
resp = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.findAll('table')
Ta = table[4].findAll('a', attrs={'href': re.compile(r'^/[a-z]')})
links = []
for t in Ta:
    links.append('https://en.wikipedia.org'+t.get('href'))

In [10]:
us_states = list(set(list(data['State[c]'])))
state_link = []
for state in us_states:
    state_link.append('https://en.wikipedia.org/wiki/'+state)

In [11]:
wiki_link = list(set(links)-set(state_link)-{'https://en.wikipedia.org/wiki/Georgia_(U.S._state)',
 'https://en.wikipedia.org/wiki/New_York_(state)',
 'https://en.wikipedia.org/wiki/Washington_(state)'})

wiki_link.sort()

In [12]:
st = set(links) - set(wiki_link)
state_links = list(st.union(set(state_link)) - {'https://en.wikipedia.org/wiki/Georgia','https://en.wikipedia.org/wiki/New_York','https://en.wikipedia.org/wiki/Washington'})

In [13]:
# Getting Abbreviations for all the states in the Table

abb = {}
for link in state_links:
    try:
        da = pd.read_html(link)
        abb[link] = da[0][da[0].columns[1]][list(da[0][da[0].columns[0]]).index('Abbreviations')][:2]
    except:
        abb[link] = 'DC'

AB = list(abb.keys())

sta_abb = {}
for i in range(len(AB)):
    sta_abb[parse_txt(AB[i],[r'[A-Z][a-zA-Z_]+'])[-1].strip('_')] = list(abb.values())[i]
    
data['Abbreviations'] = data['State[c]'].apply(lambda x: sta_abb[x])     

In [14]:
# Adding the new columns to the table
data = data.sort_values(by=['City'])
data['webpage'] = wiki_link
data = data.sort_values(by=['2018rank'])
data.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location,Abbreviations,webpage
0,1,New York City,New_York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W,NY,https://en.wikipedia.org/wiki/New_York_City
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W,CA,https://en.wikipedia.org/wiki/Los_Angeles
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W,IL,https://en.wikipedia.org/wiki/Chicago
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W,TX,https://en.wikipedia.org/wiki/Houston
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W,AZ,"https://en.wikipedia.org/wiki/Phoenix,_Arizona"


In [15]:
# function to make a dictionary of the respective race 
def make_list(lst,dfs25):
    count = []
    sites = []
    for i in range(len(dfs25)):
        for j in range(len(lst)):
            try:
                count.append(dfs25[i][dfs25[i].columns[1]][list(dfs25[i][dfs25[i].columns[0]]).index(lst[j])])
                sites.append(list(cis.keys())[i])
            except:
                pass
    dict1 = {}
    for i in range(len(sites)):
        dict1[sites[i]] = count[i]
    if len(dict1) != len(sites):
        v = list(set(cis.keys())-set(dict1.keys()).intersection(set(sites)))[0]

        dict1[str(v)] = cis[str(v)][0][cis[str(v)][0].columns[1]][0]
    return dict1

# function to create patterns from existing pattern types for different races
def scrape_patterns(websites, types):
    C = {}
    try:
        for i in range(len(websites)):
            scrape = bs.BeautifulSoup(urlopen(websites[i]))
            scrape = scrape.get_text()
            C.setdefault(websites[i], [])
            C[websites[i]].append(parse_txt(scrape, types))

    except:
        pass
    
    return C

# function to parse the values from the scraped patterns
def parse_values(websites,W,types):
    whities = {}
    for i in range(len(W)):
        try:
            whities[websites[i]] = parse_txt(str(W[i][0]),types)[0]
        except:
            whities[websites[i]] = None
    return whities

# Part II
The first part of **Part II** was probably the most challenging task of this Assignemt, mainly because the data was present in various forms and only few cities had a Demographic Table

In [16]:
websites = list(data['webpage'])

In [17]:
# getting all the demographic tables from city wikipedia 

cis = {}
for i in range(len(data)):
    try:
        df = pd.read_html(websites[i])
        for tb in df:
            if 'Demographic profile' in str(tb.columns[0]) or '1990' in str(tb.columns) or 'racial' in str(tb.columns[0]) or 'Racial composition' in str(tb.columns[0]) or 'Race' in str(tb.columns[0]) or '2010 Census' in str(tb.columns[0]) or ('White' in str(tb[tb.columns[0]]) and 'Black' in str(tb[tb.columns[0]])):
                cis.setdefault(websites[i], [])
                cis[websites[i]].append(tb)
    except:
        pass

In [18]:
frames = list(cis.values())
dfs25 = []
for i in range(len(frames)):
    dfs25.append(frames[i][0])

In [19]:
#Getting data for White Race
White_names = []
for df in dfs25:
    for i in range(len(df[df.columns[0]])): 
        if 'White' in df[df.columns[0]][i]:
                White_names.append(df[df.columns[0]][i])
                break
mylen = np.vectorize(len)
A = np.asarray(White_names)
B = mylen(A)
white_list = list(np.unique(list(A[np.where(B < 50)])))
white_dict = make_list(white_list,dfs25)

In [20]:
#Getting data for Black Race
Black_names = []
for df in dfs25:
    for i in range(len(df[df.columns[0]])): 
        if 'Black' in df[df.columns[0]][i] or 'African' in df[df.columns[0]][i]:
            if df[df.columns[0]][i] not in Black_names:
                Black_names.append(df[df.columns[0]][i])
                break
mylen = np.vectorize(len)
A = np.asarray(Black_names)
B = mylen(A)
black_list = list(A[np.where(B < 100)])
black_dict = make_list(black_list,dfs25)

In [21]:
table_sites = list(white_dict.keys())
websites2 = list(set(websites)- set(table_sites))

In [22]:
# Types of patters that I found by examining few wikipedia websites


white_types = [r'\D\d*.\d%\D\sWhite',r'\d*.\d*%\s[wW]hite',r'White[s]\s\D\d*.\d*%\D',r'White[s]\s\d*.\d*%',r'White[s]:\s\d*.\d*%',r'White:\s\d*.\d*%',r'White\sAmerican:\s\d*.\d*%',r'White\s-\s\d*.\d*%',r'\d*.\d%\swere\sWhite',r'\d*.\d\spercent\swhite',r'\d*.\d%\swhite',r'\d*.\d%\s[Nn]on-Hispanic\s[wW]hite',r'\d*.\d*%\swere\swhite',r'\D\d*.\d*%\D\snon-Hispanic\sWhite',r'\d*%\s[wW]hite',r'\d%\sWhite',r'\d\d.\d%\swas\sWhite']

In [23]:
# reproducing patterns for Black Race
b1 = []
b2 = []
b3 = []
b4 = []
b5 = []
for typ in white_types:
    b1.append(typ.replace('White','Black'))
    b2.append(typ.replace('[wW]hite','[bB]lack'))
    b3.append(typ.replace('White','African'))
    b4.append(typ.replace('[wW]hite','[aA]frican'))
    b5.append(typ.replace('white','black'))
black_types = list(set(b1).union(set(b2),set(b3),set(b4),set(b5)) - set(white_types))    

In [24]:
C = scrape_patterns(websites2, white_types)
D = scrape_patterns(websites2, black_types)

In [25]:
C1= list(C.values())
D1=list(D.values())

In [26]:
# patterns to extract values
types2 = [r'\d\d.\d%',r'\d\d.\d\d%',r'\d.\d%',r'\d.\d\d%',r'\d\d.\d\spercent',r'\d*%',r'\d.\d\spercent']

In [27]:
whities = parse_values(websites2,C1,types2)
blackies = parse_values(websites2,D1,types2)

In [28]:
C2 = list(whities.values())
D2 = list(blackies.values())

In [29]:
NONE = []
for i in range(len(websites2)):
    if C2[i] == None or D2[i] == None:
        NONE.append(websites2[i])

In [30]:
# websites that did not have any demographic data for which i will forward fill later
NONE

['https://en.wikipedia.org/wiki/Clearwater,_Florida',
 'https://en.wikipedia.org/wiki/Jurupa_Valley,_California',
 'https://en.wikipedia.org/wiki/College_Station,_Texas']

In [31]:
none_websites = {}
for website in websites:
    none_websites[website] = None    

In [32]:
newwebs1 = {**none_websites,**{**white_dict,**whities}}
newwebs2 = {**none_websites,**{**black_dict,**blackies}}

In [33]:
data['Whites'] = newwebs1.values()
data['Blacks'] = newwebs2.values()

In [34]:
#more cleaning!!!!!!!

end = data['Whites'].to_list()
White = []
for e in end:
    White.append(parse_txt(str(e),[r'\d\d.\d%',r'\d\d.\d%',r'\d\d.\d\d%',r'\d\d%']))
clean_whites = []
for w in White:
    try:
        clean_whites.append(w[0].replace('%',''))
    except:
        clean_whites.append(None)
data['White%'] = clean_whites
data['White%'] = data['White%'].astype(float) 

In [35]:
#Cleaning again

end = data['Blacks'].to_list()
Black = []
for e2 in end:
    Black.append(parse_txt(str(e2),[r'\d\d.\d%',r'\d.\d%',r'\d.\d\d%',r'\d\d%']))
clean_Blacks = []
for b in Black:
    try:
        clean_Blacks.append(b[0].replace('%',''))
    except:
        clean_Blacks.append(None)
data['Black%'] = clean_Blacks
data['Black%'] = data['Black%'].astype(float) 

In [36]:
# forward fill nan data

data = data.fillna(method='ffill')

In [37]:
data.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location,Abbreviations,webpage,Whites,Blacks,White%,Black%
0,1,New York City,New_York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W,NY,https://en.wikipedia.org/wiki/New_York_City,44.0%,25.5%,44.0,25.5
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W,CA,https://en.wikipedia.org/wiki/Los_Angeles,28.7%,9.6%,28.7,9.6
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W,IL,https://en.wikipedia.org/wiki/Chicago,44.9%,32.9%,44.9,32.9
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W,TX,https://en.wikipedia.org/wiki/Houston,25.6%,25.7%,25.6,25.7
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W,AZ,"https://en.wikipedia.org/wiki/Phoenix,_Arizona",92.3%,6.5%,92.3,6.5


## Calculating the Latitude and Longitude from DMS format

In [38]:
location = data['Location'].to_list()

In [39]:
# Function to calculate Latitude and Longitude from DegreeMinutesSeconds format

def calculate_LATandLONG(degree,minutes,seconds):
    return str(float(degree) + (float(minutes)/60) + (float(seconds)/3600))
    

In [40]:
loc1 = []
for loc in location:
    loc1.append(loc[:loc.find('W')])

In [41]:
#latitude

degree = []
minutes = []
seconds = []
for i in range(len(loc1)):
    degree.append(loc1[i].split()[0].strip('″N').split('°')[0])
    minutes.append(loc1[i].split()[0].strip('″N').split('°')[1].split('′')[0])
    seconds.append(loc1[i].split()[0].strip('″N').split('°')[1].split('′')[1])

latitude = [calculate_LATandLONG(degree[i],minutes[i],seconds[i]) for i in range(len(degree))]
data['latitude'] = [latitude[i][0:9] for i in range(len(latitude))]
data['latitude'] = data['latitude'].astype(float)    

In [42]:
#longitude

degree1 = []
minutes1 = []
seconds1 = []
for i in range(len(loc1)):
    degree1.append(loc1[i].split()[1].strip('″N').split('°')[0])
    minutes1.append(loc1[i].split()[1].strip('″N').split('°')[1].split('′')[0])
    seconds1.append(loc1[i].split()[1].strip('″N').split('°')[1].split('′')[1])
    
longitude = [calculate_LATandLONG(degree1[i],minutes1[i],seconds1[i]) for i in range(len(degree1))]
data['longitude'] = ['-'+longitude[i][:9] for i in range(len(longitude))]
data['longitude'] = data['longitude'].astype(float)

In [43]:
data.head()

,2018rank,City,State[c],2018estimate,2010Census,Change,2016 land area,2016 land area.1,2016 population density,2016 population density.1,Location,Abbreviations,webpage,Whites,Blacks,White%,Black%,latitude,longitude
0,1,New York City,New_York,8398748,8175133,+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W,NY,https://en.wikipedia.org/wiki/New_York_City,44.0%,25.5%,44.0,25.5,40.663611,-73.938611
1,2,Los Angeles,California,3990456,3792621,+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°W,CA,https://en.wikipedia.org/wiki/Los_Angeles,28.7%,9.6%,28.7,9.6,34.019444,-118.410830
2,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W,IL,https://en.wikipedia.org/wiki/Chicago,44.9%,32.9%,44.9,32.9,41.837500,-87.681666
3,4,Houston,Texas,2325502,2100263,+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W,TX,https://en.wikipedia.org/wiki/Houston,25.6%,25.7%,25.6,25.7,29.786666,-95.390833
4,5,Phoenix,Arizona,1660272,1445632,+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°W,AZ,"https://en.wikipedia.org/wiki/Phoenix,_Arizona",92.3%,6.5%,92.3,6.5,33.572222,-112.089990


In [44]:
# Arranging th columns and dropping unnecessary columns

data = data[['2018rank','City','State[c]','Abbreviations','2018estimate','2010Census','Change','2016 land area','2016 population density','Location','latitude','longitude','White%','Black%']]

In [45]:
data.columns = ['2018_Rank','City','State','Abbreviations','2018_Estimate','2010_Census','%Change','2016_land_area(sq mi)','2016_population_density(sq mi)','Location','latitude','longitude','White%','Black%']

In [46]:
remove_ = lambda x: x.replace('_', ' ')
data['State'] = data['State'].apply(remove_)

Changinig values suitable to use cases

In [47]:
data['2016_land_area(sq mi)'] = data['2016_land_area(sq mi)'].apply(lambda x: x.replace('\xa0sq\xa0mi',''))
data['2016_land_area(sq mi)'] = data['2016_land_area(sq mi)'].apply(lambda x: float(x.replace(',','')))

data['2016_population_density(sq mi)'] = data['2016_population_density(sq mi)'].apply(lambda x: x.replace('/sq\xa0mi',''))
data['2016_population_density(sq mi)'] = data['2016_population_density(sq mi)'].apply(lambda x: float(x.replace(',','')))

In [48]:
data['%Change'] = data['%Change'].apply(lambda x: x.replace('%',''))
data['%Change'] = data['%Change'].apply(lambda x: x.replace('+',''))
mean = round(data['%Change'][0:100].apply(lambda x: float(x.replace('−','-'))).astype(float).mean(),2)
data['%Change'] = data['%Change'].apply(lambda x: x.replace('[ab]',str(mean)))
data['%Change'] = data['%Change'].apply(lambda x: float(x.replace('−','-')))

In [49]:
# Making a CSV file to submit
data.to_csv('Saketh_Chityala-Topos_Assignment.csv')

# PART III

Importing plotly library to vizualize data on maps

In [50]:
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

In [51]:

Ranks = data['2018_Rank'].to_list()
Change = data['%Change'].to_list()

# Create a trace
trace = go.Scatter(
    x = Ranks,
    y = Change,
    text = data['City'],
    mode = 'markers'
)

data1 = [trace]

layout = dict(title = 'Change vs Ranks',
              yaxis = dict(title = 'Percentage Change'),
              xaxis = dict(title = 'Rank')
             )

# Plot and embed in ipython notebook!
py.iplot(data1, filename='basic-scatter')

C:\Anaconda3\envs\TFMagic\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [52]:
#color scaling
scl = [ [0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
    [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"] ]

In [54]:
data['text'] = data['City']+', '+data['2010_Census'].astype(str)

In [56]:
# Population distribution

data2 = [ go.Scattergeo(
        locationmode = 'USA-states',
        lon = data['longitude'].astype(float),
        lat = data['latitude'].astype(float),
        text = data['text'],
        mode = 'markers',
        marker = dict( 
            size = 8, 
            opacity = 0.5,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = data['2010_Census'],
            cmax = data['2010_Census'].max(),
            colorbar=dict(
                title="2010 census"
            )
        ))]
layout = dict(
        title = 'Population Distribution', 
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5        
        ),
    )
fig = go.Figure(data=data2, layout=layout )
py.iplot(fig, filename='City-Population' )

In [57]:
# calculating city population by state

state_population = {}
for i in range(len(data['State'])):
    state_population[data['State'][i]] = data[data['State']==data['State'][i]]['2010_Census'].sum()

In [58]:
# color scale
scl2 = [
    [0.0, 'rgb(188,189,220)'],
    [0.5, 'rgb(158,154,200)'],
    [0.8, 'rgb(117,107,177)'],
    [1.0, 'rgb(84,39,143)']
]

In [59]:
data['text2'] = data['State']

In [60]:
# Population distribution by state

data3 = [go.Choropleth(
    #colorscale = scl2,
    autocolorscale = True,
    locations = data['Abbreviations'],
    z = pd.DataFrame(state_population.values()),
    locationmode = 'USA-states',
    text = data['text2'],
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(255,255,255)',
            width = 2
        )),
    colorbar = go.choropleth.ColorBar(
        title = "Millions USD")
)]

layout = go.Layout(
    title = go.layout.Title(
        text = 'Population of Top cities by state'
    ),
    geo = go.layout.Geo(
        scope = 'usa',
        projection = go.layout.geo.Projection(type = 'albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)'),
)

fig = go.Figure(data = data3, layout = layout)
py.iplot(fig, filename = 'd3-cloropleth-map')

In [61]:
# Distribution of white Race across cities

data4 = [ go.Scattergeo(
        locationmode = 'USA-states',
        lon = data['longitude'].astype(float),
        lat = data['latitude'].astype(float),
        text = data['text'],
        mode = 'markers',
        marker = dict( 
            size = 8, 
            opacity = 0.5,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl2,
            cmin = 0,
            color = data['White%'],
            cmax = data['White%'].max(),
            colorbar=dict(
                title="2010 census"
            )
        ))]
layout = dict(
        title = 'White Distribution', 
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5        
        ),
    )
fig = go.Figure(data=data4, layout=layout )
py.iplot(fig, filename='white-distribution' )

In [62]:
# Distribuion on Black race across cities

data5 = [ go.Scattergeo(
        locationmode = 'USA-states',
        lon = data['longitude'].astype(float),
        lat = data['latitude'].astype(float),
        text = data['text'],
        mode = 'markers',
        marker = dict( 
            size = 8, 
            opacity = 0.5,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = scl,
            cmin = 0,
            color = data['White%'],
            cmax = data['White%'].max(),
            colorbar=dict(
                title="2010 census"
            )
        ))]
layout = dict(
        title = 'Black Distribution', 
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5        
        ),
    )
fig = go.Figure(data=data5, layout=layout )
py.iplot(fig, filename='black-distribution' )

## Final Thoughts

Thank you for the opportunity to apply for the Data engineering position at Topos. I enjoyed working on this assignment very much. I created a [github repo](https://github.com/sakethchityala/Topos-Assignment) for this assignment.

I also believe there is a better way to do some of the tasks and otimize the code to be more effieciet and intuitive. I will come back to this when I find time again.